# Smart Pointers

## `Box<T>`

* A smart pointer that allocates data on the heap.
* Provides ownership for the heap data.
* Dereferences to the data it points to.
* Useful for recursive data types and large data on the heap.

### Storing Data on Heap

In [2]:
fn main() {
    let on_heap = Box::new(5);
    println!("{}", on_heap);
}

main();

5


### Recursive Data Structures

In [3]:
enum List {
    Cons(i32, Box<List>), // Box<List> allows for recursive data structures
    Nil,
}

use List::{Cons, Nil};

fn main() {
    let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
}

main();

### Dereferencing

In [8]:
fn main() {
    let x = 5;
    let y = &x;
    let z = Box::new(x);

  assert!(5 == x);
  assert!(5 == *y);
  assert!(5 == *z);
}

main();

### How it works

In [6]:
struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        println!("Creating MyBox with some data!");
        MyBox(x)
    }
}
 
use std::ops::Deref;

impl<T> Deref for MyBox<T> {
    type Target = T;

    fn deref(&self) -> &Self::Target {
        &self.0
    }
}

impl<T> Drop for MyBox<T> {
    fn drop(&mut self) {
        println!("Dropping MyBox with some data!");
    }
}

fn main() {
    let x = 5;
    let y = MyBox::new(x);

    assert!(5 == x);
    assert!(5 == *y);
}

main();

Creating MyBox with some data!
Dropping MyBox with some data!


### Deref coercion

* Deref coercion converts a reference to a type that implements the `Deref` trait into a reference to another type. For example, deref coercion can convert `&String` to `&str` because `String` implements the `Deref` trait such that it returns `&str`

In [31]:
fn hello(name: &str) {
    println!("Hello, {}!", name);
}

fn main() {
    hello("John");
    
    let name = String::from("Jane");
    hello(&name);

    let m = MyBox::new(String::from("Rust"));
    hello(&m);
}

main();

Hello, John!
Hello, Jane!
Creating MyBox with some data!
Hello, Rust!
Dropping MyBox with some data!


## `Rc` (Reference Counted Smart Pointer)

* `Rc<T>` enables multiple ownership of the same data; `Rc<T>` keeps track of the number of references to a value which determines whether or not a value is still in use.
* `Rc<T>` is only for use in single-threaded scenarios. For multi-threaded code, `Arc<T>` should be used.
* `Rc<T>` points to immutable data. If you need mutability, put a `RefCell<T>` inside the `Rc<T>`.

In [9]:
use std::collections::HashMap;
use std::rc::Rc;

#[derive(Debug)]
struct Value(String);

impl Value {
    fn value(&self) -> &String {
        &self.0
    }
}

impl Drop for Value {
    fn drop(&mut self) {
        println!("Dropping {:?}", self);
    }
}

fn use_value(data: Rc<Value>) {
    println!("Using {:?}. RC: {}", data, Rc::strong_count(&data));
}

fn main() {
    let mut map = HashMap::<&str, Rc<Value>>::new();
    {
        let v_foo = Rc::new(Value("foo".to_string()));
        map.insert("foo", v_foo.clone());
        use_value(v_foo.clone());
        {
            let v_bar = Rc::new(Value("bar".to_string()));
            use_value(v_bar.clone());
            
            let v_inner = Rc::into_inner(v_bar);

            match v_inner {
                Some(value) => {
                    println!("v_inner - {:?} - it was only one strong reference", value);
                }
                None => {
                    println!("v_inner is None - more than one reference to it");
                }
            }
        }        
    }

    println!("{:?}", map.get("foo").unwrap());

    println!("---------------------------------");

    map.clear();

    println!("---------------------------------");
}

main();

Using Value("foo"). RC: 3
Using Value("bar"). RC: 2
v_inner - Value("bar") - it was only one strong reference
Dropping Value("bar")
Value("foo")
---------------------------------
Dropping Value("foo")
---------------------------------


## `Weak` - Breaking Reference Cycles

* `Weak<T>` is a version of `Rc<T>` that holds a non-owning reference. The object that `Weak<T>` points to is only valid if there is at least one `Rc<T>` pointing to it.
* `Weak<T>` is useful in breaking reference cycles. A reference cycle is a situation where two objects reference each other, preventing either from being cleaned up.

### Memory Leak - Reference Cycles

In [13]:
use std::cell::RefCell;
use std::fmt::{self, Display, Formatter, Debug};

struct Human {
    name: String,
    age: u32,

    partner: RefCell<Option<Rc<Human>>>,
}

impl Debug for Human {
    fn fmt(&self, f: &mut Formatter) -> fmt::Result {
        write!(f, "{} ({})", self.name, self.age)
    }
}

impl Display for Human {
    fn fmt(&self, f: &mut Formatter) -> fmt::Result {
        write!(f, "{} ({})", self.name, self.age);

        if let Some(partner) = &*self.partner.borrow() {
            write!(f, " is married to {}", partner.name)
        } else {
            write!(f, " is not married")
        }
    }
}

impl Drop for Human {
    fn drop(&mut self) {
        println!("Dropping {:?}", self);
    }
}

impl Human {
    fn new(name: &str, age: u32) -> Rc<Human> {
        Rc::new(Human {
            name: name.to_string(),
            age,
            partner: RefCell::new(None),
        })
    }
}

fn marry(h1: Rc<Human>, h2: Rc<Human>) {
    *h1.partner.borrow_mut() = Some(h2.clone());
    *h2.partner.borrow_mut() = Some(h1.clone());
}

fn main() {
    let mut john = Human::new("John", 30);
    let mut jane = Human::new("Jane", 25);

    //marry(john.clone(), jane.clone()); // this creates a reference cycle -> memory leak

    println!("John: {}", john);
    println!("Jane: {}", jane);
}

main();

John: John (30) is not married
Jane: Jane (25) is not married
Dropping Jane (25)
Dropping John (30)


### Breaking Reference Cycles with `Weak`

In [14]:
use std::rc::{Rc, Weak}
use std::cell::RefCell;
use std::fmt::{self, Display, Formatter};

struct Human {
    name: String,
    age: u32,

    partner: RefCell<Option<Weak<Human>>>,
}

impl Debug for Human {
    fn fmt(&self, f: &mut Formatter) -> fmt::Result {
        write!(f, "{} ({})", self.name, self.age)
    }
}

impl Display for Human {
    fn fmt(&self, f: &mut Formatter) -> fmt::Result {
        write!(f, "{} ({})", self.name, self.age);

        if let Some(partner) = &*self.partner.borrow() {
            if let Some(partner) = partner.upgrade() {
                write!(f, " is married to {}", partner.name)
            } else {
                write!(f, " is widow/widower")
            }
        } else {
            write!(f, " is not married")
        }
    }
}

impl Drop for Human {
    fn drop(&mut self) {
        println!("Dropping {:?}", self);
    }
}

impl Human {
    fn new(name: &str, age: u32) -> Rc<Human> {
        Rc::new(Human {
            name: name.to_string(),
            age,
            partner: RefCell::new(None),
        })
    }
}

fn marry(h1: Rc<Human>, h2: Rc<Human>) {
    *h1.partner.borrow_mut() = Some(Rc::downgrade(&h2));
    *h2.partner.borrow_mut() = Some(Rc::downgrade(&h1));
}

fn main() {
    let mut john: Rc<Human> = Human::new("John", 30);
    let mut jane: Rc<Human> = Human::new("Jane", 25);

    marry(john.clone(), jane.clone());

    println!("John: {} - Rc::strong_count({})", john, Rc::strong_count(&john));
    println!("Jane: {} - Rc::strong_count({})", jane, Rc::strong_count(&jane));

    std::mem::drop(jane);
    println!("John: {}", john);
}

main();

John: John (30) is married to Jane - Rc::strong_count(1)
Jane: Jane (25) is married to John - Rc::strong_count(1)
Dropping Jane (25)
John: John (30) is widow/widower
Dropping John (30)
